In [1]:
# this shall train neural network to recognize category / words ( total 30 ) from 1 second audio
# input audio format : format 16-bit little-endian PCM-encoded WAVE file at a 16000 sample rate

# to do : deal with padding and batching , deal with loss function

import tensorflow as tf
from __future__ import print_function
from tensorflow.contrib import rnn
import scipy.io.wavfile as wav
from python_speech_features import mfcc
import numpy as np
import time
from tensorflow.contrib.data import Dataset, Iterator


In [2]:
def decode_csv(line):
       parsed_line = tf.decode_csv(line, [[""],[""]])
       
    
       
       

       return parsed_line[0] , parsed_line[1]

In [3]:
dataset = tf.contrib.data.TextLineDataset("/home/saurabh/Documents/tf_kaggle/train.txt")
dataset=dataset.map(decode_csv)
dataset=dataset.batch(2)

In [4]:
iterator = dataset.make_initializable_iterator()

item = iterator.get_next()

In [5]:
def audio_to_mfcc(fileurl):
    rate, sig = wav.read(fileurl)
    mfcc_feat = mfcc(sig,rate)
    #d_mfcc_feat = delta(mfcc_feat, 2)
    #fbank_feat = logfbank(sig,rate)

    return mfcc_feat

In [6]:
# test above function
inputs = audio_to_mfcc('/home/saurabh/Documents/tf_orange/tf_orange/data/test.wav')

In [7]:
# function to convert audio category / label to unique number

word_list = ['bed','bird','cat','dog','down','eight','five','four','go','happy','house','left','marvin','nice','no','off','on','one','right','seven','sheila','six','stop','three','tree','two','up','wow','yes','zero','_background_noise_']
word_dictionary = {k: v for v, k in enumerate(word_list)}


def label_to_number(category):

    
    
    
  #  print (word_dictionary)
    
    for k, v in word_dictionary.items():
        if k == category:
            return v
    

    
    return -1

def number_to_label(number):
    
    for k, v in word_dictionary.items():
        if v == number:
            return k
    
     
    


In [8]:
print(label_to_number('cat'))

2


In [9]:
print(number_to_label(2))

cat


In [10]:
# Training Parameters
learning_rate = 0.001
training_steps = 20
batch_size = 2
display_step = 200


# Network Parameters

num_features = 13

timesteps = 300 # maximum timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 30   # since there are 30 categories 

In [11]:
# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_features])


In [12]:
# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([batch_size, 2 * num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [13]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell_fw = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    
    lstm_cell_bw = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    
    # Get lstm cell output
    outputs, _, _ = rnn.static_bidirectional_rnn(lstm_cell_fw, lstm_cell_bw, x,
    dtype=tf.float32)
    
    # get ony the output of last rnn timestep
    

    outputs=tf.transpose( outputs , [1, 0, 2])
    
    
    
    
    
  
   # res =  tf.matmul(outputs, weights['out']) + biases['out']
    
    return (tf.matmul(outputs, weights['out']) + biases['out'])[:,-1,:]

In [14]:
inputs = tf.placeholder(tf.float32, [None, None, num_features])
y = tf.placeholder(tf.int32 , [batch_size]) # category id for batch 

#temp = tf.one_hot(y,30)




In [15]:


logits = RNN(inputs, weights, biases)

loss_op = tf.reduce_mean( tf.nn.sparse_softmax_cross_entropy_with_logits (
    logits=logits, labels=y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

In [16]:
init = tf.global_variables_initializer()

In [21]:
with tf.Session() as sess:
    

    # Run the initializer
    sess.run(init)

    
    for curr_epoch in range(training_steps):
        
        sess.run(iterator.initializer)
        train_cost = train_ler = 0
        start = time.time()
        
       

        while True: 
            try:
                
            
                elem = sess.run(item)
          
                
                batch_inputs=elem[0]

                print(len(batch_inputs))       
                    
                temp_np_inputs = np.zeros((batch_size , timesteps , 13 ))               
            
                batch_targets = []
                
                
        
                        
                         
            
                for index,j in enumerate(elem[1]):
                    
                 #   print(elem[0][index].decode("utf-8"))
                    temp = audio_to_mfcc(elem[0][index].decode("utf-8").strip())
                    
                    difference = timesteps - temp.shape[0]
                    
                    temp_np_inputs[index] = np.pad( temp ,((0,difference),(0,0)), mode="constant")
                        
                    
                 #   print(elem[1][index].decode("utf-8").strip())
                    
                    
                  #  print(label_to_number(elem[1][index].decode("utf-8").strip()))
                    
                    batch_targets.insert(index, label_to_number(elem[1][index].decode("utf-8").strip()))
                
                feed = { inputs : temp_np_inputs , y : batch_targets }                               
                    
                print("batch ended")
                
                loss = sess.run([loss_op], feed_dict=feed)
                
                
                print(loss)
                    
                
                
            except tf.errors.OutOfRangeError:
                print("End of training dataset.")
                break
                
    
    
    
 
    
    train_cost += batch_cost*batch_size
    print(train_cost)
    

2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ende

[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]


[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]
2
batch ended
[3.3737934]


[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]
2
batch ended
[4.4539647]


KeyboardInterrupt: 